In [1]:
# Создание наборов данных
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
import math

# Загрузка обучающего набора
data_train = pd.read_csv("dataset_train_mediapipe_short.csv")

# Загрузка тестового набора
data_test = pd.read_csv("dataset_test_mediapipe_short.csv")

In [2]:
# Вычисление угла между двумя прямыми
def angle(x0, y0, x1, y1, x2, y2):
    '''
    if x1-x0!= 0:
        k1 = (y1 - y0) / (x1 - x0)
        if x2-x0!=0:
            k2 = (y2 - y0) / (x2 - x0)
        tg_angle = abs((k1 - k2) / (1 + k1 * k2))
        return math.atan(tg_angle)
    return math.pi/2
    '''
    calc_angle = math.degrees(math.atan2(y2 - y0, x2 - x0) - math.atan2(y1 - y0, x1 - x0))
    if calc_angle < 0:
        calc_angle += 360
    return calc_angle
#print(angle(0, 0, 0, 1, 2, 0))

In [3]:
Angle_mediaPipe = ['RightElbowAngle', 'LeftElbowAngle', 'RightHandAngle', 'LeftHandAngle', 'RightKneeAngle', 'LeftKneeAngle',
                   'RightLegAngle', 'LeftLegAngle', 
                  ]
Key_Points_MediaPipe = ['Nose', 'LeftEyeInnerCorner', 'LeftEye', 'LefEyeOuterCorner', 
              'RightEyeInnerCorner', 'RightEye', 'RightEyeOuterCorner',
              'LeftEar', 'RightEar', 'MouthLeftCorner', 'MouthRightCorner', 'LeftShoulder', 'RightShoulder',
              'LeftElbow', 'RightElbow', 'LeftWrist', 'RightWrist', 'LeftLittleFinger', 'RightLittleFinger',
              'LeftIndexFinger', 'RightIndexFinger', 'LeftThumb', 'RightThumb', 
              'LeftHip', 'RightHip', 'LeftKnee', 'RightKnee', 'LeftAnkle', 'RightAnkle',
              'LeftHeel', 'RightHeel', 'TheIndexFingerOfTheLeftFoot', 'TheIndexFingerOfTheRightFoot'
]

In [4]:
# Создание пустого датафрейма
def init_data(angle_name):
    data = []
    data.append("FileName")
    for p in angle_name:
        data.append(p)
    data.append("target")
    data = pd.DataFrame(columns = data) 
    return data

In [5]:
# Создание датасета каркасных моделей (набора углов и метки - названия позы)
def create_df_angle_pos(angle_name, df, name_file):
    data = init_data(angle_name)
    target = []
    for i in range(len(df)):
        temp = []
        temp += [df.loc[i]['FileName']]
        temp += [angle(df.loc[i]['RightElbow_x'], df.loc[i]['RightElbow_y'], 
                        df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'], 
                        df.loc[i]['RightWrist_x'], df.loc[i]['RightWrist_y'])]
        temp += [angle(df.loc[i]['LeftElbow_x'], df.loc[i]['LeftElbow_y'], 
                        df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'], 
                        df.loc[i]['LeftWrist_x'], df.loc[i]['LeftWrist_y'])] 
        temp += [angle(df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'],
                        df.loc[i]['RightElbow_x'], df.loc[i]['RightElbow_y'], 
                        df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'])]
        temp += [angle(df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'],
                        df.loc[i]['LeftElbow_x'], df.loc[i]['LeftElbow_y'], 
                        df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'])]
        temp += [angle(df.loc[i]['RightKnee_x'], df.loc[i]['RightKnee_y'],
                        df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'], 
                        df.loc[i]['RightAnkle_x'], df.loc[i]['RightAnkle_y'])]
        temp += [angle(df.loc[i]['LeftKnee_x'], df.loc[i]['LeftKnee_y'],
                        df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'], 
                        df.loc[i]['LeftAnkle_x'], df.loc[i]['LeftAnkle_y'])]
        temp += [angle(df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'],
                       df.loc[i]['RightKnee_x'], df.loc[i]['RightKnee_y'],
                       df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'])]
        temp += [angle(df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'],
                       df.loc[i]['LeftKnee_x'], df.loc[i]['LeftKnee_y'],
                       df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'])]
        temp += [df.loc[i]['target']]
        data.loc[i] = temp # Запись в датафрейм

    data.to_csv(name_file) # Запись датафрейма углов в файл
    return data

In [6]:
# Построение датафрейма угловых скелетных моделей обучающего набора
data_tr_angle = create_df_angle_pos(Angle_mediaPipe, data_train, 'dataset_train_mediapipe_angle.csv')

print(data_tr_angle.info())
data_tr_angle.head()

<class 'pandas.core.frame.DataFrame'>
Index: 963 entries, 0 to 962
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FileName         963 non-null    object 
 1   RightElbowAngle  963 non-null    float64
 2   LeftElbowAngle   963 non-null    float64
 3   RightHandAngle   963 non-null    float64
 4   LeftHandAngle    963 non-null    float64
 5   RightKneeAngle   963 non-null    float64
 6   LeftKneeAngle    963 non-null    float64
 7   RightLegAngle    963 non-null    float64
 8   LeftLegAngle     963 non-null    float64
 9   target           963 non-null    object 
dtypes: float64(8), object(2)
memory usage: 82.8+ KB
None


,FileName,RightElbowAngle,LeftElbowAngle,RightHandAngle,LeftHandAngle,RightKneeAngle,LeftKneeAngle,RightLegAngle,LeftLegAngle,target
0,DATASET/TRAIN/downdog/00000128.jpg,191.787354,192.985691,171.522721,174.424631,183.073871,182.089113,53.048521,52.923798,downdog
1,DATASET/TRAIN/downdog/00000129.jpg,195.833544,194.272616,182.271665,177.248445,178.552442,173.804520,49.930037,46.776987,downdog
2,DATASET/TRAIN/downdog/00000130.jpg,195.182323,185.816359,190.819733,189.593200,146.384282,143.254167,46.260924,47.007644,downdog
3,DATASET/TRAIN/downdog/00000131.jpg,168.997754,195.806537,168.638008,193.399865,182.239773,176.440491,63.133218,50.617992,downdog
4,DATASET/TRAIN/downdog/00000132.png,202.544370,243.190369,191.474216,273.568657,133.988707,193.207867,327.018874,14.938305,downdog


In [7]:
# Построение датафрейма угловых скелетных моделей тестового набора
data_te_angle = create_df_angle_pos(Angle_mediaPipe, data_test, 'dataset_test_mediapipe_angle.csv')

print(data_te_angle.info())
data_te_angle.head()

<class 'pandas.core.frame.DataFrame'>
Index: 435 entries, 0 to 434
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FileName         435 non-null    object 
 1   RightElbowAngle  435 non-null    float64
 2   LeftElbowAngle   435 non-null    float64
 3   RightHandAngle   435 non-null    float64
 4   LeftHandAngle    435 non-null    float64
 5   RightKneeAngle   435 non-null    float64
 6   LeftKneeAngle    435 non-null    float64
 7   RightLegAngle    435 non-null    float64
 8   LeftLegAngle     435 non-null    float64
 9   target           435 non-null    object 
dtypes: float64(8), object(2)
memory usage: 37.4+ KB
None


,FileName,RightElbowAngle,LeftElbowAngle,RightHandAngle,LeftHandAngle,RightKneeAngle,LeftKneeAngle,RightLegAngle,LeftLegAngle,target
0,DATASET/TEST/downdog/00000000.jpg,192.838657,194.158524,184.110470,187.249455,184.005206,184.364423,51.714598,52.936687,downdog
1,DATASET/TEST/downdog/00000001.JPG,177.356228,203.647163,176.025895,198.499156,180.701341,164.790156,1.675774,37.355014,downdog
2,DATASET/TEST/downdog/00000002.jpg,177.068454,196.346337,143.168979,184.657213,194.561761,179.841567,329.064105,352.240960,downdog
3,DATASET/TEST/downdog/00000003.jpg,175.574746,223.236828,158.236635,237.285138,192.632002,205.565995,351.273518,8.933032,downdog
4,DATASET/TEST/downdog/00000004.jpg,186.614473,185.702877,183.677236,181.745721,147.685569,184.386152,33.991283,56.170744,downdog
